# Crime Prediction
## - Author: Frank Serafine
---

**GOAL:** Train various time series machine learning models to predict crime activity over a year, given ten years of history.

**SOURCE:** FBI's [Crime Data Explorer](https://crime-data-explorer.fr.cloud.gov/pages/downloads)

**TRIMMED INCIDENT-LEVEL DATA SET:** [Crime_Pred.csv](https://drive.google.com/file/d/1nM8-DCRj2GA8FxOi04btjwks0gxKAQom/view?usp=sharing)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('Crime_Pred.csv')
df_altdrop = pd.read_csv('crimepred_altdrop.csv')
df.head()

In [ ]:
# Percentage of total observations that are null:

df.isna().sum()/df.shape[0]*100

# 4.6% of crime incidents did not have an hour logged.
# 34.6% of crime incidents did not log the age of the offender.
# 27.4% of crime incidents did not have an age listed for the victim, but this does not always indicate an omission:
# Crimes against an institution or against society do not have a specific victim and thus no victim age.

In [ ]:
df.info()

In [ ]:
df['Date'] = pd.to_datetime(df['Date']) 

In [ ]:
dupe_data = df.duplicated(keep=False)
df.loc[dupe_data, :]

In [ ]:
df.nunique()

In [ ]:
df_altdrop = df_altdrop.drop_duplicates(keep='first')

In [ ]:
df_altdrop.isna().sum()/df.shape[0]*100

In [ ]:
# from sklearn.preprocessing import OneHotEncoder

# # Instantiate the OneHotEncoder
# ohe = OneHotEncoder()

# # Fit the OneHotEncoder to the  column and transform
# # It expects a 2D array, so we first convert the column into a DataFrame
# offense = pd.DataFrame(df['Offense'])
# encoded = ohe.fit_transform(subcategory)
# encoded